<a href="https://colab.research.google.com/github/diegomrodrigues/llm/blob/main/BERT_Fine_Tunning_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers datasets huggingface_hub tensorboardX accelerate --upgrade

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from huggingface_hub import notebook_login
from huggingface_hub import HFSummaryWriter

In [ ]:
notebook_login()

In [ ]:
from transformers import TrainerCallback
import time

class HFMetricsCallback(TrainerCallback):
    def __init__(self, logger):
        self.logger = logger

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        self.logger.add_scalar("loss", metrics["eval_loss"])
        self.logger.add_scalar("accuracy", metrics["eval_accuracy"])
        self.logger.add_scalar("f1", metrics["eval_f1"])
        self.logger.add_scalar("precision", metrics["eval_precision"])
        self.logger.add_scalar("recall", metrics["eval_recall"])


In [ ]:
def load_and_preprocess_data(tokenizer):
    # Load the IMDB dataset
    imdb = load_dataset("imdb")

    # Preprocess the dataset
    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

    # Encode the dataset
    imdb_encoded = imdb.map(preprocess_function, batched=True, remove_columns=["text"])

    return imdb_encoded

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

def train_model(imdb_encoded, tokenizer, logger):
    # Create a data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=5,
        evaluation_strategy='steps',
        eval_steps=5,
        save_strategy='steps',
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        report_to=["tensorboard"]
    )

    # Load the BERT model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=imdb_encoded['train'],
        eval_dataset=imdb_encoded['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[HFMetricsCallback(logger)]
    )

    # Train the model
    trainer.train()

    # Save the trained model
    trainer.save_model("./trained_model")

def main():
    # Logger to HF
    logger = HFSummaryWriter(repo_id="test_hf_logger", commit_every=0.5)

    # Load the BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Load and preprocess the data
    imdb_encoded = load_and_preprocess_data(tokenizer)

    # Train the model
    train_model(imdb_encoded, tokenizer, logger)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_experimental.py:58: UserWarning: 'HFSummaryWriter' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
5,0.694400,0.708247,0.451720,0.549749,0.466366,0.669440
10,0.718100,0.706384,0.458120,0.547906,0.470026,0.656720
15,0.727200,0.703484,0.464040,0.521857,0.471043,0.584960
20,0.693500,0.699465,0.486680,0.470477,0.485812,0.456080


{'eval_loss': 0.7082474231719971, 'eval_accuracy': 0.45172, 'eval_f1': 0.5497487107052524, 'eval_precision': 0.46636571364877666, 'eval_recall': 0.66944, 'eval_runtime': 858.6954, 'eval_samples_per_second': 29.114, 'eval_steps_per_second': 0.455, 'epoch': 0.003198976327575176}
{'eval_loss': 0.7063844799995422, 'eval_accuracy': 0.45812, 'eval_f1': 0.5479058902052395, 'eval_precision': 0.470025765817349, 'eval_recall': 0.65672, 'eval_runtime': 860.5678, 'eval_samples_per_second': 29.051, 'eval_steps_per_second': 0.454, 'epoch': 0.006397952655150352}
{'eval_loss': 0.7034838795661926, 'eval_accuracy': 0.46404, 'eval_f1': 0.5218570459979303, 'eval_precision': 0.47104296849835725, 'eval_recall': 0.58496, 'eval_runtime': 859.3058, 'eval_samples_per_second': 29.093, 'eval_steps_per_second': 0.455, 'epoch': 0.009596928982725527}
{'eval_loss': 0.6994649767875671, 'eval_accuracy': 0.48668, 'eval_f1': 0.47047658345368265, 'eval_precision': 0.4858116744780571, 'eval_recall': 0.45608, 'eval_runtime'

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
5,0.694400,0.708247,0.451720,0.549749,0.466366,0.669440
10,0.718100,0.706384,0.458120,0.547906,0.470026,0.656720
15,0.727200,0.703484,0.464040,0.521857,0.471043,0.584960
20,0.693500,0.699465,0.486680,0.470477,0.485812,0.456080


KeyboardInterrupt: 

KeyboardInterrupt: 